In [5]:
import json
from math import ceil
from pathlib import Path
from datasets import load_dataset
import pandas as pd

In [14]:
df = pd.read_csv("../dpo/data/0514university_dpo_kor_data.csv")
df = df.drop(['Unnamed: 0'], axis=1)
len(df)

398

In [19]:
df

,input,accept,reject,context
0,모든 모듈에 대해 각 학생이 등록한 총 모듈 수를 계산하고 학생과 등록한 총 모듈 ...,"SELECT id_student, COUNT(*) AS total_modules_r...","SELECT sr.id_student, COUNT(sr.code_module) AS...",CREATE TABLE studentregistration (code_module ...
1,지역명과 지역별 Pass 등급을 받은 학생들의 평균 학점,"SELECT region, AVG(studied_credits) AS avg_cre...","SELECT region, AVG(studied_credits) AS avg_cre...",CREATE TABLE studentinfo (code_module varchar(...
2,가장 많은 학점을 이수한 모듈의 학생번호 및 최종 성적을 조회,"SELECT sr.id_student, si.final_result FROM stu...","SELECT sr.id_student, si.final_result FROM stu...",CREATE TABLE studentinfo (code_module varchar(...
3,등록한 학생들 중 VLE 자료와 상호 작용한 학생들의 연령대별 평균 학점이 80보다...,"SELECT s.age_band, AVG(studied_credits) AS avg...","SELECT studentinfo.age_band, AVG(studentinfo.s...",CREATE TABLE studentinfo (code_module varchar(...
4,각 모듈 프레젠테이션에 등록한 학생 중 VLE 자료를 이용한 학생의 평균 상호 작용...,"SELECT sr.code_module, sr.code_presentation, A...","SELECT studentregistration.code_module, studen...",CREATE TABLE studentinfo (code_module varchar(...
...,...,...,...,...
393,각 지역의 지역명과 학생들의 평균 나이를 조회,"SELECT region, AVG(CAST(SUBSTRING(age_band, 1,...","SELECT region, AVG(age_band) AS avg_age FROM s...",CREATE TABLE studentinfo (code_module varchar(...
394,평균 클릭 수 보다 높은 클릭 수의 학생들의 평균 성적을 구해줘,SELECT AVG(studentassessment.score) AS average...,SELECT AVG(studentassessment.score) AS average...,CREATE TABLE studentassessment (id_assessment ...
395,평가 결과가 'Distinction'이고 모든 평가 유형에서 평균 점수가 80보다 ...,SELECT COUNT(*) AS distinction_count FROM ( SE...,SELECT COUNT(DISTINCT studentinfo.id_student) ...,CREATE TABLE studentassessment (id_assessment ...
396,각 모듈 프레젠테이션에서 가장 높은 점수를 받은 학생의 최종 결과를 식별하고 모듈 ...,"SELECT sr.code_module, sr.code_presentation, s...","SELECT sr.code_module, sr.code_presentation, s...",CREATE TABLE studentinfo (code_module varchar(...


In [20]:
def dataframe_to_json(csv:str, save_path:str):
    df = pd.read_csv(csv)
    df = df[['input', 'accept', 'reject', 'context']]
    
    with open(save_path, "w") as f:
        for row in df.iterrows():
            newitem = {
                "input": row[1]['input'],
                "accept": row[1]['accept'],
                "reject": row[1]['reject'],
                "context": row[1]['context'],
            }
            f.write(json.dumps(newitem) + "\n")

def get_train_val_splits(
    data_dir: str,
    val_ratio: float = 0,
    # dpo_ratio: float = 0.2,
    seed: int = 42,
    shuffle: bool = True,
):
     
    def load_jsonl(data_dir):
        data_path = Path(data_dir).as_posix()
        data = load_dataset("json", data_files=data_path)
        return data

    data = load_jsonl(data_dir)
    num_samples = len(data["train"])
    val_set_size = ceil(val_ratio * num_samples)

    train_data_split = data["train"].train_test_split(
        # test_size=val_set_size,
        shuffle=shuffle, seed=seed
    )
    
    return train_data_split["train"].shuffle()
    # return train_data_split["train"].shuffle(), train_data_split["test"].shuffle()
    
    # dpo_set_size = ceil(dpo_ratio * len(train_data_split['test']))
    # dpo_data_split = train_data_split["test"].train_test_split(
    #     test_size=dpo_set_size,  shuffle=shuffle, seed=seed
    # )
    # return train_data_split["train"].shuffle(), dpo_data_split["train"].shuffle(), dpo_data_split["test"].shuffle()

def save_jsonl(data_dicts, out_path):
    with open(out_path, "w") as fp:
        for data_dict in data_dicts:
            fp.write(json.dumps(data_dict) + "\n")

In [21]:
data_csv = '../dpo/data/0514university_dpo_kor_data.csv'
json_name = '../dpo/data/university_orpo_data.jsonl'

In [22]:
dataframe_to_json(csv=data_csv, save_path=json_name)
sft_train_data = get_train_val_splits(data_dir=json_name) # 추후 excel 변환
# save_jsonl(sft_train_data, "../dpo/data/0514university_orpo_data.jsonl")
# save_jsonl(test_data, "../dpo/data/0514university_test_row.jsonl")

Generating train split: 0 examples [00:00, ? examples/s]

In [10]:
len(sft_train_data)

80

In [11]:
len(test_data)

NameError: name 'test_data' is not defined